In [21]:
import sys
sys.path.append("..")

In [22]:

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from satreeverify.rf_utils import *

In [23]:
data = load_breast_cancer()
feature_names = data["feature_names"]

In [24]:
X = data['data']
y = data["target"] == 1
feature_names = data["feature_names"]
ntrees = 50
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)

In [25]:
clf.score(X, y)

0.9736379613356766

In [26]:
all_thresh = get_ens_thresh(clf)
data_ = disc_ens_data(X, all_thresh)
print(data_.shape)
data_.head()

(569, 295)


,25_0.3684999942779541,28_0.37904998660087585,25_0.25665000081062317,4_0.10064999759197235,7_0.049230000004172325,21_31.664999961853027,27_0.1419999971985817,7_0.05615000054240227,23_690.6000061035156,6_0.062425000593066216,...,26_0.3678999990224838,5_0.08531500026583672,12_6.540499925613403,3_694.1499938964844,3_694.5,0_15.320000171661377,0_15.275000095367432,0_15.025000095367432,22_115.45000076293945,13_24.225000381469727
0,True,True,True,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,True,False,True,True,True,True,...,False,False,False,True,True,True,True,True,True,True
2,True,False,True,True,True,False,True,True,True,True,...,True,True,False,True,True,True,True,True,True,True
3,True,True,True,True,True,False,True,True,False,True,...,True,True,False,False,False,False,False,False,False,True
4,False,False,False,False,True,False,True,True,True,True,...,True,True,False,True,True,True,True,True,True,True


In [27]:
var_x = create_var_x(all_thresh)

# Soft Solution

In [28]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.81837229, 0.18162771]])

In [29]:
s, c_weights = soft_attack(clf, sample, epsilon, var_x)

False
sat


In [30]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.823526428560553

In [31]:
clf.predict_proba(sample)[0][1]

0.18162771064496697

In [32]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
27,0.198600,0.121490,0.388269,"[0.11134999990463257, 0.12150000035762787]"
25,0.409900,0.256592,0.374013,"[0.19869999587535858, 0.25665000081062317]"
13,34.660000,21.807545,0.370815,"[19.355000495910645, 21.8100004196167]"
24,0.186200,0.118836,0.361783,"[0.10486000031232834, 0.11885000020265579]"
8,0.192500,0.126274,0.344033,"[-inf, 0.12639999762177467]"
7,0.096010,0.063652,0.337026,"[0.06080000102519989, 0.06365500018000603]"
6,0.207100,0.137325,0.336916,"[0.11205000057816505, 0.13735000044107437]"
4,0.139800,0.094334,0.325220,"[0.08851999789476395, 0.0943400003015995]"
3,530.200000,694.150344,0.309224,"[694.1499938964844, 694.5]"
5,0.176500,0.124874,0.292498,"[0.09899000078439713, 0.12489999830722809]"


In [33]:
clf.predict_proba(x_adv_sample)[0][1]

0.823526428560553

# Hard Solution

In [34]:
epsilon = 0.3

nbits = 8

index = 105
sample = X[index:index+1, :]

In [35]:
s, c_weights, seq_num = hard_attack(clf, sample, epsilon, var_x, nbits)

False
sat


In [36]:
clf.predict_proba(sample)[0][1]

0.18162771064496697

In [37]:
get_value(s.model(), seq_num, nbits , ntrees)

0.5005490196078431

In [38]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.5020974772408908

In [39]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
5,0.176500,0.124874,0.292498,"[0.09899000078439713, 0.12489999830722809]"
27,0.198600,0.143699,0.276442,"[0.1423499956727028, 0.1437000036239624]"
29,0.140500,0.102343,0.271579,"[0.09544999897480011, 0.10235000029206276]"
1,15.560000,19.624605,0.261221,"[19.605000495910645, inf]"
24,0.186200,0.137699,0.260479,"[0.13655000180006027, 0.13769999891519547]"
25,0.409900,0.305002,0.255912,"[0.25665000081062317, 0.30505000054836273]"
3,530.200000,660.415894,0.245598,"[660.3999938964844, 676.2999877929688]"
28,0.314700,0.387837,0.232404,"[0.3874499946832657, inf]"
0,13.110000,10.749240,0.180073,"[-inf, 10.759999752044678]"
23,827.200000,690.595606,0.165141,"[686.2000122070312, 690.6000061035156]"


In [40]:
clf.predict_proba(x_adv_sample)[0][1]

0.5020974772408908